In [1]:
import os

In [2]:
%pwd

'c:\\Users\\konde\\Music\\projects\\Kidney Cancer Detection\\Kidney-Cancer-Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\konde\\Music\\projects\\Kidney Cancer Detection\\Kidney-Cancer-Detection'

In [7]:
import dagshub
dagshub.init(repo_owner='jayasampreethreddy', repo_name='Kidney-Cancer-Detection', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\konde\AppData\Local\Programs\Python\Python312\Lib\site-packages\rich\live.py:229: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=866f264e-976a-450f-8619-5f9a4a9c5289&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=73a096453c763a0d1727a3ce8f1ef7fa88a3ee665b4f5a7f9d3db43cbabdc3fb




Accessing as jayasampreethreddy

Initialized MLflow to track repo "jayasampreethreddy/Kidney-Cancer-Detection"

Repository jayasampreethreddy/Kidney-Cancer-Detection initialized!

2024/09/05 12:31:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run persistent-swan-948 at: https://dagshub.com/jayasampreethreddy/Kidney-Cancer-Detection.mlflow/#/experiments/0/runs/03b4a6b1f54a4580befc2949a5f7abc8.
2024/09/05 12:31:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/jayasampreethreddy/Kidney-Cancer-Detection.mlflow/#/experiments/0.


In [8]:
import tensorflow as tf

In [16]:
model = tf.keras.models.load_model("artifacts/training/model.h5")
print(model)

<Functional name=functional, built=True>


In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [17]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/jayasampreethreddy/Kidney-Cancer-Detection.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [18]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [19]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != "file":

                
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [20]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.


c:\Users\konde\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 1.0000 - loss: 0.0342


2024/09/05 12:38:13 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Successfully registered model 'VGG16Model'.
2024/09/05 12:38:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
2024/09/05 12:38:47 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/09/05 12:38:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run welcoming-mare-213 at: https://dagshub.com/jayasampreethreddy/Kidney-Cancer-Detection.mlflow/#/experiments/0/runs/333c014a9af6420788b04eab0ebfc7e3.
2024/09/05 12:38:48 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/jayasampreethreddy/Kidney-Cancer-Detection.mlflow/#/experiments/0.
